In [1]:
import pandas as pd

In [2]:
Tags = pd.read_csv("Filtered_tags.csv")
Questions = pd.read_csv("Combined.csv")

C:\Users\beboz\.conda\envs\computervision\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tag_map = {
    "html5":"html",
    "iphone":"ios",
    "ruby-on-rails":"ruby",
    "ruby-on-rails-3":"ruby",
    "asp.net-mvc":"asp.net",
    "xcode":"ios",
    "vba":"excel",
    "python-2.7":"python",
    "css3":"css",
    "excel-vba":"excel",
    "sql-server-2008":"sql-server",
    "visual-studio-2010":"visual-studio",
    "google-maps":"google",
    "google-chrome":"google",
    "jsp":"java",
    "python-3.x":"python"
}

In [4]:
def reset_tags(tag):
    try:
        return tag_map[tag]
    except:
        return tag

In [5]:
Tags["Tag"] = Tags["Tag"].apply(lambda x : reset_tags(x))

In [6]:
Tags["Tag"].value_counts()

javascript      124155
java            119524
c#              101186
php              98808
android          90659
                 ...  
wcf               4480
unit-testing      4437
sockets           4421
symfony2          4228
tsql              4158
Name: Tag, Length: 85, dtype: int64

In [7]:
Tags.shape

(1755529, 3)

In [8]:
Tags = Tags[["Id","Tag"]]

In [9]:
Tags = Tags.drop_duplicates()

In [10]:
Tags.shape

(1699701, 2)

In [11]:
Tags = Tags.groupby("Id",as_index=False).agg(lambda x: ','.join(x))

In [12]:
Tags.head()

,Id,Tag
0,120,"sql,asp.net"
1,180,algorithm
2,260,"c#,.net"
3,330,"c++,class"
4,470,".net,web-services"


In [13]:
Tags["Tag"] = Tags["Tag"].apply(lambda x:x.split(','))

In [14]:
data = pd.merge(Questions,Tags,on="Id")

In [15]:
data.head()

,Unnamed: 0,Id,Title,Body,Tag
0,2,120,ASP.NET Site Maps,have anyone get experience create sql-based as...,"[sql, asp.net]"
1,3,180,Function for creating color wheels,be something i 've pseudo-solved many time nev...,[algorithm]
2,4,260,Adding scripting functionality to .NET applica...,i have little game write in c use database as ...,"[c#, .net]"
3,5,330,Should I use nested classes in this case?,i be work on collection of class use for video...,"[c++, class]"
4,6,470,Homegrown consumption of web services,i 've be write few web service for .net app no...,"[.net, web-services]"


In [16]:
data = data[["Body","Tag"]]

In [17]:
Body = data["Body"]
Tag = data["Tag"]

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

In [19]:
binarizer = MultiLabelBinarizer()
Tag = binarizer.fit_transform(Tag)

In [20]:
Tag = pd.DataFrame(Tag,columns = binarizer.classes_)

In [27]:
tfidf = TfidfVectorizer(max_features = 2000,ngram_range=(1,2), stop_words = 'english')
Body_features = tfidf.fit_transform(Body)


In [ ]:
type(Body_features)

scipy.sparse._csr.csr_matrix

In [ ]:
# col_to_drop = []
# for col in binarizer.classes_:
#     if (Tag[col].value_counts()[1] < 1200):
#         col_to_drop.append(col)

In [ ]:
# Tag.drop(columns = col_to_drop,inplace=True)

In [ ]:
#x -> Body_features  y -> Tag

In [22]:
# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

In [23]:
#helper functions 
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits = 10, n_repeats = 85, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
      # prepare data
      X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
      y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
      # define model
      model = get_model(n_inputs, n_outputs)
      # fit model
      model.fit(X_train, y_train, verbose = 0, epochs = 100)
      # make a prediction on the test set
      yhat = model.predict(X_test)
      # round probabilities to class labels
      yhat = yhat.round()
      # calculate accuracy
      acc = accuracy_score(y_test, yhat)
      # store result
      print('>%.3f' % acc)
      results.append(acc)
    return results

In [29]:
#x -> Body_features  y -> Tag
results = evaluate_model(features, Tag)

#testing the model 
# row = [3, 3, 6, 7, 8, 2, 11, 11, 1, 3]
# newX = asarray([row])
# yhat = model.predict(newX)
# print('Predicted: %s' % yhat[0])

>0.383
>0.382
>0.391


MemoryError: Unable to allocate 2.92 GiB for an array with shape (195810, 2000) and data type float64

In [28]:
features = pd.DataFrame.sparse.from_spmatrix(Body_features)

In [ ]:
features.head()


,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.022277,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.030836,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(features.shape[0])

217567


In [30]:
import transformers

ModuleNotFoundError: No module named 'transformers'